In [1]:
import pandas as pd

In [4]:
products = pd.DataFrame({
    "product_id": [1, 2, 3, 4, 5],
    "product_name": [
        "Leetcode Solutions",
        "Jewels of Stringology",
        "HP",
        "Lenovo",
        "Leetcode Kit"
    ],
    "product_category": ["Book", "Book", "Laptop", "Laptop", "T-shirt"]
})

# Orders table
orders = pd.DataFrame({
    "product_id": [1,1,2,2,3,3,3,4,4,5,5,5],
    "order_date": [
        "2020-02-05","2020-02-10","2020-01-18","2020-02-11",
        "2020-02-17","2020-02-24","2020-03-01",
        "2020-03-04","2020-03-04",
        "2020-02-25","2020-02-27","2020-03-01"
    ],
    "unit": [60,70,30,80,2,3,20,30,60,50,50,50]
})

orders["order_date"] = pd.to_datetime(orders["order_date"])


In [7]:
mask = orders['order_date'].dt.to_period("M") == "2020-02"

In [9]:
df = orders[mask].groupby(by = "product_id", as_index = False)['unit'].sum()
df

,product_id,unit
0,1,130
1,2,80
2,3,5
3,5,100


In [10]:
mask2 = (df.unit >= 100)

In [12]:
new = df[mask2]
new

,product_id,unit
0,1,130
3,5,100


In [13]:
new.merge(products , on = "product_id", how = "left")

,product_id,unit,product_name,product_category
0,1,130,Leetcode Solutions,Book
1,5,100,Leetcode Kit,T-shirt


In [44]:
activity = pd.DataFrame({
    "player_id": [1, 1, 2, 3, 3],
    "device_id": [2, 2, 3, 1, 4],
    "event_date": [
        "2016-03-01",
        "2016-03-02",
        "2017-06-25",
        "2016-03-02",
        "2018-07-03"
    ],
    "games_played": [5, 6, 1, 0, 5]
})
activity["event_date"] = pd.to_datetime(activity["event_date"])
activity

,player_id,device_id,event_date,games_played
0,1,2,2016-03-01,5
1,1,2,2016-03-02,6
2,2,3,2017-06-25,1
3,3,1,2016-03-02,0
4,3,4,2018-07-03,5


In [45]:
tot_players = activity.player_id.nunique()
tot_players

3

In [50]:
activity["start_date"] = activity.groupby(by = "player_id", as_index = False)["event_date"].transform("min")
activity

,player_id,device_id,event_date,games_played,start_date
0,1,2,2016-03-01,5,2016-03-01
1,1,2,2016-03-02,6,2016-03-01
2,2,3,2017-06-25,1,2017-06-25
3,3,1,2016-03-02,0,2016-03-02
4,3,4,2018-07-03,5,2016-03-02


In [52]:
activity["diff"] = activity["event_date"] - activity["start_date"]
activity

,player_id,device_id,event_date,games_played,start_date,diff
0,1,2,2016-03-01,5,2016-03-01,0 days
1,1,2,2016-03-02,6,2016-03-01,1 days
2,2,3,2017-06-25,1,2017-06-25,0 days
3,3,1,2016-03-02,0,2016-03-02,0 days
4,3,4,2018-07-03,5,2016-03-02,853 days


In [53]:
mask = (activity["diff"] == "1 days")

In [58]:
 log_back = activity[mask]["player_id"].nunique()

In [59]:
round(log_back/tot_players, 2)

0.33

In [ ]:
df = merged2.groupby(by="request_at").apply(
            lambda x: ((x["status"].str.startswith("cancelled")) & (x["banned_client"] == "No")).sum() / x[
                "banned_client"].str.startswith("No").sum()).rename("Cancellation Rate").reset_index()


In [86]:
s = "LVIII"

In [91]:
rom = {'I': 1,
       'V': 5,
       'X' : 10,
       'L': 50,
       'C' : 100,
       'D' : 500,
       'M' : 1000}
rom

{'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000}